<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/PJH/0501_Ensemble_(bagging).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import pandas as pd
import numpy as np
import re
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

In [ ]:
# 데이터 파일 로드
df = pd.read_csv('/content/drive/MyDrive/Last.csv')
df = df.drop(columns=["vul", "Unnamed: 0"])
df['code'] = df['code'].apply(lambda x: re.sub(r'/\*.*?\*/', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'//.*?\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#include <.*?>\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#include \".*?\"\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'#define .*?\n', '', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'[\t ]+', ' ', x))
df['code'] = df['code'].apply(lambda x: re.sub(r'\n\s*\n', '\n', x))
df.to_csv("/content/preprocessed_data.csv", index=False)

In [ ]:
# 데이터셋 생성
dataset = DatasetDict({
    'train': load_dataset('csv', data_files="/content/preprocessed_data.csv")['train']
})

model_names = [
    "microsoft/graphcodebert-base",
    "microsoft/codebert-base-mlm",
    "microsoft/unixcoder-base"
]

models = [AutoModelForSequenceClassification.from_pretrained(name, num_labels=11) for name in model_names]
tokenizers = [AutoTokenizer.from_pretrained(name) for name in model_names]


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/938k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/444k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
# 메트릭 계산 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"f1": f1_score(labels, predictions, average='weighted')}

In [ ]:
# K-Fold 교차 검증 설정
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

all_predictions = []
for model, tokenizer in zip(models, tokenizers):
    model_predictions = []
    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset['train'])):
        print(f"Training fold {fold+1}/{k}")
        train_data = dataset['train'].select(train_idx)
        val_data = dataset['train'].select(val_idx)

        tokenized_train_data = train_data.map(lambda x: tokenizer(x['code'], padding="max_length", truncation=True, max_length=512), batched=True)
        tokenized_val_data = val_data.map(lambda x: tokenizer(x['code'], padding="max_length", truncation=True, max_length=512), batched=True)

        training_args = TrainingArguments(
            output_dir=f'./results/{model.__class__.__name__}_fold_{fold}',
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=64,
            num_train_epochs=5,
            weight_decay=0.01,
            load_best_model_at_end=True,
            save_total_limit=1,  # 최신 모델만 저장
            metric_for_best_model='f1',
            greater_is_better=True
        )

Training fold 1/5


Map:   0%|          | 0/14892 [00:00<?, ? examples/s]

Map:   0%|          | 0/3724 [00:00<?, ? examples/s]

Training fold 2/5


Map:   0%|          | 0/14893 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/14893 [00:00<?, ? examples/s]

Map:   0%|          | 0/3723 [00:00<?, ? examples/s]

Training fold 5/5


Map:   0%|          | 0/14893 [00:00<?, ? examples/s]

Map:   0%|          | 0/3723 [00:00<?, ? examples/s]

In [ ]:
        early_stopping_callback = EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.001
        )

In [ ]:
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_train_data,
            eval_dataset=tokenized_val_data,
            compute_metrics=compute_metrics,
            callbacks=[early_stopping_callback]
        )
        trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# 예측 수집
        predictions = trainer.predict(tokenized_val_data).predictions.argmax(-1)
        model_predictions.append(predictions)

In [ ]:
# 모델 예측 결합
    final_predictions = np.mean(model_predictions, axis=0)
    all_predictions.append(final_predictions)


In [ ]:
# 모든 모델의 예측 결합
final_prediction = np.mean(all_predictions, axis=0).round().astype(int)